# Implementação do Algoritmo Simplex 
versão Julia

In [1]:
# Bibliotecas
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Funcao para imprimir tableu
def print_tableau(tableau):
    lins = len(tableau)
    cols = len(tableau[0])
    
    header =  ['x' + str(i+1) for i in range(cols-1)]
    print('           ' + ''.join(f'{col:>8}' for col in header))
    print('              ' + '-' * (7 * cols))
    
    print('F.O. ' + ''.join(f'{val:8.2f}' for val in tableau[0]))
    for i in range(1,lins-1):
        print(f'x{tableau[-1][i-1]+1} = ' + ''.join(f'{val:8.2f}' for val in tableau[i]))
    
    print()

# Imprime somente a matriz A
def print_matrix(matrix):
    for linha in matrix:
        print(''.join(f'{val:8.2f}' for val in linha))

# Fase 1: obter solução básica viável

In [3]:
# Procura alguma coluna da matriz Identidade
def eh_basica(coluna):
    return sum(coluna) == 1 and len([c for c in coluna if c == 0]) == len(coluna) - 1

# Retorna o indice onde esta "1"
def get_one(vector):
    for i in range(len(vector)):
        if vector[i] == 1:
            return i

# Funcao que guarda o indice das variaveis basicas
# associado a coluna da matriz identidade
def basicas(A):
    id = [0] * len(A)       # coluna da identidade
    i_B = [-1] * len(A)     # indice da variavel
    colunas = np.array(A).T
    for coluna in range(0, len(colunas)):
        if eh_basica(colunas[coluna]) and id[get_one(colunas[coluna])] == 0:
            i_B[get_one(colunas[coluna])]= coluna
            id[get_one(colunas[coluna])] = 1
    return id, i_B
# OBS: quando escreve basicas(A)[0] --> acessa quais colunas
#                     basicas(A)[1] --> acessa os indices (-1) das basicas

def fase1(c, A, b):
    # Verifica se é necessario adicionar variaveis artificiais
    tem_sol_basica = any(x == -1 for x in basicas(A)[1])
    if not tem_sol_basica:
        d = False
        return c, A, b, d
 
    # Se nessesario multiplicar a linha por -1 para que
    # os termos independentes sejam nao negativos
    for linha in range(0, len(b)):
        if b[linha] < 0:
            A[linha] = -A[linha]
            b[linha] = -b[linha]   

    # Custos da nova funcao
    c1 = [0] * len(c)    # zero para as variaveis originais
    id = basicas(A)[0]
    for i in range(0, len(id)):
        if id[i] == 0:
            coluna = [0] * len(id)
            coluna[i] = 1
            for linha in range(0, len(A)):
                A[linha].append(coluna[linha])
            c1.append(1)  # um para as variaveis artificiais
    
    # se precisou adicionar variaveis
    d = True
    
    return c1, A, b, d

A fase 1 deve retornar um Tableau !!

Usamos somente as colunas:

    - da funcao objetivo e valores das basicas
    - das variaveis originais
    
OBS: se a solucao encontrada for degenerada entao excluimos a linha de zeros

In [4]:
# Reorganiza o tableau
def resultado_fase1(tableau):
    # mantem as colunas das variaveis originais
    qtdd_colunas = len(tableau[-1])
    
    novo_A = []
    novo_b = []
    
    # mantem as linhas das variaveis basicas originais
    for i, xB in enumerate(tableau[-1]):
        if xB < qtdd_colunas:
            novo_A.append(tableau[i+1][1:qtdd_colunas+1])
            novo_b.append(tableau[i+1][0])
    return novo_A, novo_b

# Fase 2: Aplicar Método Simplex

## Tableau Inicial 

Estamos assumindo que para montar o Tableau está sendo passada uma matriz A que contenha todas as colunas da identidade necessarias para obter uma solucao basica viável.

OBS: garantimos isso pois antes todos os problemas passam pela fase 1


In [5]:
# Monta o Tableau Inicial
def tableau_inicial(c, A, b):
    linhas_tableau = []
    
    # guarda as basicas
    identidade, indice_basicas = basicas(A)
    
    # funcao objetivo e custos reduzidos
    funcao_obj = (-1) * sum(b)
    custos = []
    for j in range(len(A[0])):
        if j not in indice_basicas:
            c_j = 0
            for i in range(len(A)):
                c_j += A[i][j]
            custos.append((-1)*c_j)
        else: 
            custos.append(0)
    
    linhas_tableau.append([funcao_obj]+custos)  # linha 0

    # Concatena linhas de A com linhas de b
    for i in range(len(b)):
        linha = [b[i]] + A[i]
        #print("The linha is: ",linha)
        linhas_tableau.append(linha)

    print("+++++++++++++++")

    # indice das variaveis basicas
    linhas_tableau.append(indice_basicas)  # ultima linha
    return linhas_tableau
        

In [6]:
# Verifica se tem algum custo reduzido negativo
def nao_otima(tableau):
    return any(c < 0 for c in tableau[0][1:])

# Define elemento pivo
def posicao_pivo(tableau):
    custos = tableau[0]
    # Escolhe o primeiro custo reduzido negativo
    # "xj entra na base"
    for j in range(1,len(custos)):
        if custos[j] < 0:
            coluna_pivo = j
            break

    theta = []
    for linha_i in range (1,len(tableau)-1):
        if tableau[linha_i][coluna_pivo] <= 0:
            theta.append(math.inf)
        else:
            theta.append(tableau[linha_i][0] / tableau[linha_i][coluna_pivo])
        
    # Escolhe o menor indice das variaveis que podem sair da base
    # soma um porque estamos ignorando a linha de custos
    linha_pivo = theta.index(min(theta)) + 1
    
    return linha_pivo, coluna_pivo

# Atualiza o tableau com base no pivo encontrado
def atualiza_tableau(tableau, pivo):
    i, j = pivo
    valor_pivo = tableau[i][j]
    
    # Divide a linha do pivo para que o valor do pivo seja 1
    tableau[i] = np.array(tableau[i]) / valor_pivo
    
    # Atualiza os valores de dentro do tableau
    # ou seja, zera os outros elementos da coluna
    for linha in range (len(tableau)-1):
        if linha != i:
            multiplicador = (-1) * tableau[linha][j]
            tableau[linha] = multiplicador*tableau[i] + tableau[linha]
   
    # Atualiza variaveis básicas
    tableau[-1][i-1] = j-1 

    return tableau

def otima(tableau):
    # inicia com todos valendo zero
    solucoes = [0] * len(tableau[0]-1)
    
    # coloca o valor das basicas na posicao correta da solucao
    for i, xB in enumerate(tableau[-1]):
        solucoes[xB] = tableau[i+1][0]
        
    return solucoes        
    

In [7]:
# SIMPLEX para resolver FASE 1
def simplex(c, A, b,d):
    tableau = tableau_inicial(c, A, b)
    #print("Tableau inicial: ")
    #print_tableau(tableau)
    
    while nao_otima(tableau):
        pivo = posicao_pivo(tableau)
        tableau = atualiza_tableau(tableau, pivo)
        #print("Tableau atualizado: ")
        #print_tableau(tableau)
    
    if d:
        novo_A, novo_b = resultado_fase1(tableau)
        return novo_A, novo_b
    
    else:
        return A, otima(tableau)    

# Testando exemplo dos slides aula 23 #
Solucão básica viável para iniciar o simplex:
(x1,x2,x3,x4) = (1 ; 1/2 ; 1/3 ; 0) e zero nas variaveis a mais

Problema: A tem linhas LD --> sol. encontrada na fase 1 é degenerada

In [8]:
# Coeficientes da função objetivo
c = [1, 1, 1, 0]

# Matriz de coeficientes das restrições
A = [
    [1, 2, 3, 0],
    [-1, 2, 6, 0],
    [0, 4, 9, 0],
    [0, 0, 3, 1]
]

# Vetor de termos independentes das restrições
b = [3, 2, 5, 1]

In [9]:
def get_solution_simplex(c, A, b):

    c1, A1, b1, d = fase1(c, A, b)

    # Caso seja necessario usar variaveis artificiais
    if d:
        print("Simplex FASE 1: ")
        A2, b2 = simplex(c1,A1,b1,d)
        print("Resultado da fase 1: ",b2)
        
        # Convertendo tudo para mesmo formato que o simplex
        A2 = [[float(x) for x in row] for row in A2]
        b2 = [float(x) for x in b2] 
        fA, fb = simplex(c, A2, b2,0)
        print("Resultado da fase 2: ",fb)
    else:
        A2, b2 = simplex(c1, A1, b1,d)
        print("Nao Foi nessesario usar a fase 1!!")
        print("Resultado da fase 2: ", b2)

In [10]:
get_solution_simplex(c, A, b)


Simplex FASE 1: 
+++++++++++++++
Resultado da fase 1:  [np.float64(1.0), np.float64(0.5), np.float64(0.3333333333333333)]
+++++++++++++++
Resultado da fase 2:  [np.float64(0.5), np.float64(1.25), 0, np.float64(1.0), 0]


In [11]:

# Objective function coefficients (minimization, so we use negative values)
c = [-2, -1, 0, 0]  # coefficients for [y₁, y₂, s₁, s₂]

# Constraint matrix
A = [
    [-1, 1, 1, 0],  
    [1, 2, 0, 1]   
]

# RHS values
b = [-1, 7]

get_solution_simplex(c, A, b)

+++++++++++++++
Nao Foi nessesario usar a fase 1!!
Resultado da fase 2:  [np.float64(3.0), np.float64(2.0), 0, 0, 0]
